### Imported the necessary libraries

In [7]:
import bs4
import requests 
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

### Defined the download function. All the images are stored in the 'Images generated' folder

In [8]:
def download_image(url, folder_name, num1):
    
    # write image to file
    reponse = requests.get(url)
    if reponse.status_code == 200:
        with open(os.path.join(folder_name, str(num1) + ".jpg"), 'wb') as file:
            file.write(reponse.content)
    
# create a directory to save images
folder_name = r'C:\Users\birud\OneDrive - iitgn.ac.in\github\Image Classifier\WoC_6.0_Checkpoint_2\dog_dalmatian_images' # name whatever you want
if not os.path.isdir(folder_name):
    os.makedirs(folder_name)

### Opened a chrome window

In [3]:
chromeDriverPath = r'C:\Users\birud\OneDrive - iitgn.ac.in\github\Image Classifier\Srep1 - Data Collection using web scraper\Drivers\chromedriver.exe'

# Create a ChromeOptions object instead of using the deprecated DesiredCapabilities
chrome_options = Options()

# Add any additional options you need, for example:
# chrome_options.add_argument("--headless")

# Use ChromeService instead of the deprecated Service
service = ChromeService(chromeDriverPath)

# Pass the chrome_options and service to the webdriver.Chrome constructor
driver = webdriver.Chrome(service=service, options=chrome_options)


### Enter the URL below and scroll the newly opened window to its bottom most point

In [9]:
inp = input("Enter url: ", )
search_url = inp
driver.get(search_url)

Enter url: https://www.google.com/search?q=dalmatian+images&source=lnms&tbm=isch


### Enter the number of images to be generated

In [10]:
x = int(input("Enter the number of images required : ", ))

Enter the number of images required : 140


### Below is the code, run it.

In [11]:
a = input("Press any key and hit enter to start : ")
# Scrolling all the way up
driver.execute_script("window.scrollTo(0, 0);")
scroll_distance = 60 # You can change to improve performance

page_html = driver.page_source
pageSoup = bs4.BeautifulSoup(page_html, 'html.parser')
containers = pageSoup.findAll('div', {'class':"isv-r PNCib ViTmJb BUooTd"}) # Choose the class based on the search_url

len_containers = len(containers)
print("Found %s image containers"%(len_containers))

# selector for first image in search_url : #islrg > div.islrc > div:nth-child(2)
# Xpath for first image in search_url : //*[@id="islrg"]/div[1]/div[1]

# Xpaths that should be ignored :
# 1) //*[@id="islrg"]/div[1]/div[25]
# 2) //*[@id="islrg"]/div[1]/div[50]

count_images_generated = 0
missed_list = []
timeout_seconds = 10
timeout = 10

# Loop 1
for i in range(1, x + 1):

    if count_images_generated == x:
        break
        
    if (len(missed_list) == 20):
        break
        
#     if i == 8 or i == 9 or i == 10:# or i == 32:
#         continue
        
    xPath = '//*[@id="islrg"]/div[1]/div[%s]' % (i)

    try:
        # Grabbing the URL of the small preview image
        previewImageXPath = '//*[@id="islrg"]/div[1]/div[%s]/a[1]/div[1]/img' % (i)
        
        # Adding a timeout for finding the preview image element
        previewImageElement = WebDriverWait(driver, 0.05).until(
            EC.presence_of_element_located((By.XPATH, previewImageXPath))
        )
        
        previewImageURL = previewImageElement.get_attribute("src")
        # clicking on the image container
        driver.find_element('xpath', xPath).click()

    except Exception as e:
        print(f"Xpath has changed, wait untill we collect the missed tag: {i}")
        driver.execute_script(f"window.scrollTo(0, {i*scroll_distance});")
        time.sleep(0.5)
        missed_list.append(i)
        continue

    # Starting a while True loop to wait until we see that the URL inside the large image view is different from the preview one
    timeStarted = time.time()
    while True:
        try:
            imageElement = WebDriverWait(driver, timeout).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="Sva75c"]/div[2]/div[2]/div[2]/div[2]/c-wiz/div/div/div/div/div[3]/div[1]/a/img[1]'))
            )
            imageURL = imageElement.get_attribute('src')

            if imageURL != previewImageURL:
                break
            else:
                # making a timeout if the full res image can't be loaded
                currentTime = time.time()

                if currentTime - timeStarted > timeout_seconds:
                    print("Timeout! Exiting the loop.")
                    break

        except Exception as e:
            print(f"Error: {e}. Couldn't retrieve full res image URL. Continuing to the next one.")
            break  # Continue to the next iteration if an error occurs

    # Try to download the image
    try:
        download_image(imageURL, folder_name, count_images_generated)
        print("Downloaded element %s. URL: %s" % (i, imageURL))
        count_images_generated += 1

    except Exception as e:
        print(f"Error: {e}. Couldn't download the image {i}, continuing downloading the next one.")

    time.sleep(1)  # Add a small delay to avoid overloading the server

# Loop 2
time.sleep(3)

for i in missed_list:
    if count_images_generated == x:
        break

    missed_count = 0
    for j in range(1, x + 1):
#         if j == 7 or j == 8 or j == 9 or j == 10 or j == 11:
#             continue
            
        k = i*j
        
        if count_images_generated == x:
            break
            
        if missed_count >= 5: # You can change to improve performance
#             driver.refresh()
            driver.execute_script(f"window.scrollTo(0, {k*scroll_distance});")
            time.sleep(2)

            page_html = driver.page_source
            pageSoup = bs4.BeautifulSoup(page_html, 'html.parser')
            containers = pageSoup.findAll('div', {'class':"isv-r PNCib ViTmJb BUooTd"}) # Choose the class based on the search_url
            break

        xPath = '//*[@id="islrg"]/div[1]/div[%s]/div[%s]' % (i, j)

        try:
#             driver.execute_script(f"window.scrollTo(0, {(i+j)*scroll_distance});")
            previewImageXPath = '//*[@id="islrg"]/div[1]/div[%s]/div[%s]/a[1]/div[1]/img' % (i, j)
            # Adding a timeout for finding the preview image element
            previewImageElement = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, previewImageXPath))
            )
            
            previewImageURL = previewImageElement.get_attribute("src")

            # clicking on the image container
            driver.find_element('xpath', xPath).click()

        except Exception as e:
#             print(f"Error: {e}. Couldn't process image [{i}, {j}]. Continuing to the next one.")
            print(f"Couldn't find element [{i}, {j}]. Checking if Xpath has changed")
#             driver.execute_script(f"window.scrollTo(0, {k*scroll_distance});")
            time.sleep(1)
            missed_count += 1
            continue

        # Starting a while True loop to wait until we see that the URL inside the large image view is different from the preview one
        timeStarted = time.time()
        while True:
            try:
                imageElement = WebDriverWait(driver, timeout).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="Sva75c"]/div[2]/div[2]/div[2]/div[2]/c-wiz/div/div/div/div/div[3]/div[1]/a/img[1]'))
                )
                imageURL = imageElement.get_attribute('src')

                if imageURL != previewImageURL:
                    break
                else:
                    # making a timeout if the full res image can't be loaded
                    currentTime = time.time()

                    if currentTime - timeStarted > timeout_seconds:
                        print("Timeout! Exiting the loop.")
                        break

            except Exception as e:
                print(f"Error: {e}. Couldn't retrieve full res image URL. Continuing to the next one.")
                break  # Continue to the next iteration if an error occurs

        # Try to download the image
        try:
            download_image(imageURL, folder_name, count_images_generated)
            print("Downloaded element [%s, %s]. URL: %s" % (i, j, imageURL))
            count_images_generated += 1
            missed_count = 0

        except Exception as e:
            print(f"Error: {e}. Couldn't download the image [{i}, {j}], continuing downloading the next one.")

        time.sleep(1)  # Add a small delay to avoid overloading the server

if x == count_images_generated:
    print("Target Reached")
else:
    print("There is some issue, scroll down the page and run code again")


Press any key and hit enter to start : a
Found 48 image containers
Downloaded element 1. URL: https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Sun_Dog_Dalmatian.jpg/640px-Sun_Dog_Dalmatian.jpg
Downloaded element 2. URL: https://cdn.britannica.com/47/236047-050-F06BFC5E/Dalmatian-dog.jpg
Downloaded element 3. URL: https://cdn.britannica.com/49/236049-050-AEEBDD63/Dalmation-dog-brown-and-white.jpg
Downloaded element 4. URL: https://i.pinimg.com/originals/88/8a/52/888a520875f01246e31a30bbf33602af.jpg
Downloaded element 5. URL: https://media.gettyimages.com/id/175453491/photo/dalmatian.jpg?s=612x612&w=gi&k=20&c=Pt-MkhFlvrPzGncQkrcTWdMShhT1kPgoJHmJ4wun0D8=
Downloaded element 6. URL: https://upload.wikimedia.org/wikipedia/commons/1/15/Dalmatiner_2.jpg
Downloaded element 7. URL: https://www.shutterstock.com/image-photo/dalmatian-puppies-spotted-dog-puppy-600nw-2225467117.jpg
Downloaded element 8. URL: https://www.akc.org/wp-content/uploads/2017/11/Dalmatian-On-White-01.jpg
Downloaded

Couldn't find element [50, 2]. Checking if Xpath has changed
Couldn't find element [50, 3]. Checking if Xpath has changed
Couldn't find element [50, 4]. Checking if Xpath has changed
Couldn't find element [50, 5]. Checking if Xpath has changed
Couldn't find element [50, 6]. Checking if Xpath has changed
Downloaded element [51, 1]. URL: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQnVNgeDeWAwqSpti6qCeGgsC-lJMZKK1KJiw&usqp=CAU
Downloaded element [51, 2]. URL: https://www.creativefabrica.com/wp-content/uploads/2023/01/08/Cute-Small-Baby-Dalmatian-Dog-Puppy-56813762-1.png
Downloaded element [51, 3]. URL: https://i.redd.it/tjftf8n39mu21.jpg
Downloaded element [51, 4]. URL: https://assets.wfcdn.com/im/21000558/compr-r85/1613/161320267/dalmatian-mom-nuzzling-her-baby-photo-art-print-black-wood-framed-poster-20x14.jpg
Downloaded element [51, 5]. URL: https://as1.ftcdn.net/v2/jpg/03/21/37/30/1000_F_321373073_Ld3xg5i1QkmcEVQmgiEtuhZdvzE5BnoN.jpg
Timeout! Exiting the loop.
Downloaded ele

Downloaded element [51, 47]. URL: https://p7.hiclipart.com/preview/704/757/299/dalmatian-dog-the-hundred-and-one-dalmatians-perdita-puppy-the-101-dalmatians-musical-baby-dalmation-cliparts.jpg
Downloaded element [51, 48]. URL: https://cdn11.bigcommerce.com/s-cy4lua1xoh/images/stencil/1280x1280/products/14574/99560/beanie-babies-ty-beanie-baby-bo-the-dalmatian-dog-7-inch-plush-2002-new-w-tags__21131.1659848851.jpg?c=1
Downloaded element [51, 49]. URL: https://images.nightcafe.studio/jobs/RyCjQ3Z8dhvUgZNHtU52/RyCjQ3Z8dhvUgZNHtU52--1--52i8a.jpg?tr=w-1600,c-at_max
Couldn't find element [51, 50]. Checking if Xpath has changed
Downloaded element [51, 51]. URL: https://media.sciencephoto.com/image/z9320092/800wm/Z9320092.jpg
Downloaded element [51, 52]. URL: https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Dalmatian_puppy.JPG/150px-Dalmatian_puppy.JPG
Downloaded element [51, 53]. URL: https://www.warrenphotographic.co.uk/photography/bigs/03550-baby-girl-with-Dalmatian-father-and-pup-